In [1]:
#Imports
import cssutils
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [3]:
#Store info from page
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
#NASA Mars News
#Grab all article titles and teasers
Nasa_Articles = soup.find('ul', class_='item_list')
Nasa_Mars_Titles_html = Nasa_Articles.find_all('div', class_='content_title')
Nasa_Mars_Content_html = Nasa_Articles.find_all('div', class_='article_teaser_body')

Nasa_Mars_Titles = [article.text for article in Nasa_Mars_Titles_html]
Nasa_Mars_Content = [article.text for article in Nasa_Mars_Content_html]

In [5]:
#Grabing the first
Nasa_Mars_Newest_Title = Nasa_Mars_Titles[0]
Nasa_Mars_Newest_Content = Nasa_Mars_Content[0]

In [6]:
#JPL Mars Space Images - Featured Image
url ='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [7]:
Nasa_Image = soup.find('div', class_='carousel_items')
Nasa_Image_Style = Nasa_Image.find('article')['style']
Styles = cssutils.parseStyle(Nasa_Image_Style)
Back_Image_Arguement = Styles['background-image']
End_Path = Back_Image_Arguement.replace('url(', '').replace(')', '')
Featured_Image_url = 'https://www.jpl.nasa.gov' + End_Path

In [8]:
#Mars Facts
url ='https://space-facts.com/mars/'
tables = pd.read_html(url)

In [9]:
Mars_Facts_DF = tables[0]
Mars_Facts_DF = Mars_Facts_DF.rename(columns={0: "Parameters", 1: "Values"})
Mars_Facts_DF = Mars_Facts_DF.set_index('Parameters')
Mars_Facts_DF

,Values
Parameters,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
html = Mars_Facts_DF.to_html()

In [50]:
#Mars Hemispheres
url ='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
Grab_Hemisphere_Links = browser.links.find_by_partial_text('Hemisphere Enhanced')

In [51]:
Hemisphere_list =[]
for link in range(len(browser.links.find_by_partial_text('Hemisphere Enhanced'))):
    browser.visit(url)
    Grab_Hemisphere_Links = browser.links.find_by_partial_text('Hemisphere Enhanced')
    image_text = Grab_Hemisphere_Links[link].text
    Grab_Hemisphere_Links[link].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image = soup.find_all('img', class_='wide-image')
    for image in image:
        image_url = 'https://astrogeology.usgs.gov' + image['src']
        Hemisphere_list.append({"title": image_text, "img_src": image_url})
        image['src']
    browser.visit(url)

In [62]:
for item in Hemisphere_list:
    print(item['img_src'])

https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
